# Testing classfiers on the 10k pairs dataset

## Loading in data

In [1]:
!pip install pandas 

import pandas as pd


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [68]:
# Load in data

dtype_dict_all = {
    'conversation_id': 'object',
    'id': 'object',
    'author_id': 'object',
    'referenced_tweets_id': 'object',
    'in_reply_to_user_id': 'object',
    'PNR': 'object'
}

pairs = pd.read_csv('/Users/idahelenedencker/Desktop/STANDBY_Ida/Creating dataset of reference tweets/10k_twitter_pairs.csv', dtype=dtype_dict_all )

pairs

,conversation_id,lang,created_at,id,text,author_id,replied_to_reply_count,referenced_tweets_id,in_reply_to_user_id,PNR,...,A/R,fasttext_cos_sim_hate_sentence,fasttext_cos_sim_prosocial_sentence,tweeter_username,tweeter_name,pair_num,type,like_n,retweet_n,quote_n
0,1036583587242549248,da,2018-09-03 21:04:41,1036721666628444160,@frkomo Jeg siger det vel strengt taget bare t...,1666088336,1.0,1036721302692917250,148061237,1311570613,...,0-0,0.762309,0.959726,NaN,NaN,1,reply,1,0,0
1,1036583587242549248,da,2018-09-03 21:03:14,1036721302692917250,"@MonbergSF Sig det til spillerforeningen, som ...",148061237,NaN,NaN,1666088336,NaN,...,NaN,NaN,NaN,frkomo,Sarah Agerklint,1,tweet,1,0,0
2,899548260863488002,da,2017-08-21 12:10:42,899604671375052801,"@PeterHuggler Had alt det, du vil. Men du skal...",547416021,1.0,899603561323081729,3301029597,1405772015,...,0-0,0.836744,0.958750,NaN,NaN,2,reply,0,0,0
3,899548260863488002,da,2017-08-21 12:06:17,899603561323081729,@brianweichardt Jeg hader den her slags: Du gå...,3301029597,NaN,NaN,547416021,NaN,...,NaN,NaN,NaN,PeterHuggler,Peter Huggler,2,tweet,1,0,0
4,1345496479583113217,da,2021-01-03 12:38:16,1345711074407014400,@nielscallesoe @Heunicke Din første indvending...,87923613,1.0,1345524516311748608,23341699,0908801199,...,0-0,0.746267,0.957824,NaN,NaN,3,reply,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1430605648878784513,da,2021-08-25 19:00:13,1430605952722604045,@PSkipperEL Træk den røde linje nu.\n\n#VæltMe...,2345329126,NaN,NaN,611076925,NaN,...,NaN,NaN,NaN,M_K_Petersen,Michael K. Petersen,9998,tweet,16,0,0
19996,822180433681088513,da,2017-01-19 20:55:03,822185628620226560,@sllaursen har svært ved at afgøre om jeg ægte...,546738902,1.0,822180433681088513,43529378,1007802400,...,0-0,0.886249,0.839126,NaN,NaN,9999,reply,3,0,0
19997,822180433681088513,da,2017-01-19 20:34:25,822180433681088513,"Når et sponsorat rammer ESPN, er man kommet ok...",43529378,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,steen,Steen Laursen,9999,tweet,13,1,0
19998,953648851721826304,da,2018-01-17 18:22:52,953694124049420288,@emilnielsen @pelledragsted Tak skal du have. ...,26210681,0.0,953655858675757057,19391295,2010772527,...,0-1,0.886244,0.897734,NaN,NaN,10000,reply,10,0,0


In [3]:
pairs.dtypes

conversation_id                         object
lang                                    object
created_at                              object
id                                      object
text                                    object
author_id                               object
replied_to_reply_count                 float64
referenced_tweets_id                    object
in_reply_to_user_id                     object
PNR                                     object
surveyXact_externke                     object
non_unique_twitter_author_id           float64
started_survey                         float64
rec-nition                             float64
attack                                 float64
A/R                                     object
fasttext_cos_sim_hate_sentence         float64
fasttext_cos_sim_prosocial_sentence    float64
tweeter_username                        object
tweeter_name                            object
pair_num                                 int64
type         

In [ ]:
!pip install --upgrade transformers
!pip install --upgrade tqdm

In [ ]:
#Make a test df to work with 

pairs_small= pairs.head(2000)

In [69]:
# make a translated text column

from tqdm import tqdm
from transformers import MarianMTModel, MarianTokenizer

# Load the model and tokenizer
model_name = 'Helsinki-NLP/opus-mt-da-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

def translate(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", padding=True)
    # Generate translation
    translated_tokens = model.generate(**inputs)
    # Decode the tokens to text
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return translated_text


# List to store the scores
list = []

# Iterate over each text
for text in (tqdm(pairs_small['text'], desc= "translating")) :
    result = translate(text)
    # Append to the list
    list.append(result)

# Add the scores as a new column in the DataFrame
pairs_small['translated'] = list
pairs_small


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
translating: 100%|████████████████████████| 2000/2000 [2:41:37<00:00,  4.85s/it]
/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_20768/4113161718.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pairs_small['translated'] = list


,conversation_id,lang,created_at,id,text,author_id,replied_to_reply_count,referenced_tweets_id,in_reply_to_user_id,PNR,...,fasttext_cos_sim_hate_sentence,fasttext_cos_sim_prosocial_sentence,tweeter_username,tweeter_name,pair_num,type,like_n,retweet_n,quote_n,translated
0,1036583587242549248,da,2018-09-03 21:04:41,1036721666628444160,@frkomo Jeg siger det vel strengt taget bare t...,1666088336,1.0,1036721302692917250,148061237,1311570613,...,0.762309,0.959726,NaN,NaN,1,reply,1,0,0,@frkomo I guess I'm saying it strictly just to...
1,1036583587242549248,da,2018-09-03 21:03:14,1036721302692917250,"@MonbergSF Sig det til spillerforeningen, som ...",148061237,NaN,NaN,1666088336,NaN,...,NaN,NaN,frkomo,Sarah Agerklint,1,tweet,1,0,0,"@MonbergSF Tell it to the gaming association, ..."
2,899548260863488002,da,2017-08-21 12:10:42,899604671375052801,"@PeterHuggler Had alt det, du vil. Men du skal...",547416021,1.0,899603561323081729,3301029597,1405772015,...,0.836744,0.958750,NaN,NaN,2,reply,0,0,0,@PeterHuggler Had everything you want. But don...
3,899548260863488002,da,2017-08-21 12:06:17,899603561323081729,@brianweichardt Jeg hader den her slags: Du gå...,3301029597,NaN,NaN,547416021,NaN,...,NaN,NaN,PeterHuggler,Peter Huggler,2,tweet,1,0,0,@brianweichardt I hate this kind of thing: You...
4,1345496479583113217,da,2021-01-03 12:38:16,1345711074407014400,@nielscallesoe @Heunicke Din første indvending...,87923613,1.0,1345524516311748608,23341699,0908801199,...,0.746267,0.957824,NaN,NaN,3,reply,1,0,0,@nielsallesoe @Heunicke Your first objection m...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1285495153202024448,da,2020-07-21 10:22:06,1285520419529863168,@SimonStoerup @perlysholt Hm. Man ville som ud...,27626050,NaN,NaN,383396359,NaN,...,NaN,NaN,nielsfez,Niels Pedersen,998,tweet,2,0,0,@SimonStoerup @perlysholt Hm. You would be abl...
1996,1174395406719102976,da,2019-09-18 19:10:18,1174400270710820870,@R4nd4hl @khoenge Sjovt du synes netop Hønge b...,861057936,0.0,1174395406719102976,72823792,1508892043,...,0.736094,0.932785,NaN,NaN,999,reply,2,0,0,@R4nd4hl @khoenge Funny you think just Hønge s...
1997,1174395406719102976,da,2019-09-18 18:50:58,1174395406719102976,Detektor har undersøgt det: @khoenge talte usa...,72823792,NaN,NaN,NaN,NaN,...,NaN,NaN,R4nd4hl,Randahl Fink,999,tweet,20,3,0,Detector has examined it: @khoenge spoke untru...
1998,1481298462238982144,da,2022-01-12 19:25:54,1481346717513662464,"@ReneAndersenDK Forskellen er, at ham her også...",805874425988087811,1.0,1481345545721495554,4776986009,NaN,...,0.787638,0.932775,NaN,NaN,1000,reply,2,0,0,@ReneAndersenDK The difference is that this gu...


In [60]:
print(pairs_small[['text', 'translated']].head(10))

                                                text  \
0  @frkomo Jeg siger det vel strengt taget bare t...   
1  @MonbergSF Sig det til spillerforeningen, som ...   
2  @PeterHuggler Had alt det, du vil. Men du skal...   
3  @brianweichardt Jeg hader den her slags: Du gå...   
4  @nielscallesoe @Heunicke Din første indvending...   
5  @stinuslindgreen @Heunicke I min optik, er det...   
6  @RasmusMalver @radikale Det er jeg ked af. Jeg...   
7  Indtil for en time siden var @stinuslindgreen ...   
8  @larskohler @BEsbensen @PiaOlsen Lidt, men jeg...   
9  @ThomasMonbergSF Vi løber vist i ring!... Ha e...   

                                          translated  
0  @frkomo I guess I'm saying it strictly just to...  
1  @MonbergSF Tell it to the gaming association, ...  
2  @PeterHuggler Had everything you want. But don...  
3  @brianweichardt I hate this kind of thing: You...  
4  @nielsallesoe @Heunicke Your first objection m...  
5  @stinuslindgreen @Heunicke In my optics, it is... 

## Zero shot learning

In zero shot learning a pre-trained model is used to predict the give a probability on any given label using embeddings, even though the label(s) has not been part of the training data

### facebook/bart-large-mnli model

In [4]:
from transformers import pipeline


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [5]:
pipe_1 = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)

In [6]:
candidate_labels = ['counterspeech', 'hate', 'neutral tone']

sequence_to_classify_1 = "i see your point but i dont think it is very constructive"
sequence_to_classify_2 = "jeg forstår hvad du mener, men din kommentar er ikke særlig konstruktiv"
sequence_to_classify_3 = pairs['text'][3]
sequence_to_classify_4 = "today i went for a walk outside with my dog, nice sunny weather"

pipe_1(sequence_to_classify_2, candidate_labels)

#maybe also good on danish, test on danish also, make a new column


{'sequence': 'jeg forstår hvad du mener, men din kommentar er ikke særlig konstruktiv',
 'labels': ['counterspeech', 'neutral tone', 'hate'],
 'scores': [0.819023072719574, 0.17399725317955017, 0.006979694124311209]}

In [73]:
# on test data (translated)

# Candidate labels
candidate_labels = ['counterspeech', 'hate', 'neutral tone']

# List to store the scores
counterspeech_scores = []
hate_scores = []
neutral_scores = []

# Iterate over each text
for text in (tqdm(pairs_small['translated'], desc= "Extracting scores")) :
    result = pipe_1(text, candidate_labels)
    # Extract the scores
    counterspeech_score = dict(zip(result['labels'], result['scores']))['counterspeech']
    hate_score = dict(zip(result['labels'], result['scores']))['hate']
    neutral_score = dict(zip(result['labels'], result['scores']))['neutral tone']
    # Append the scores to the lists
    counterspeech_scores.append(counterspeech_score)
    hate_scores.append(hate_score)
    neutral_scores.append(neutral_score)

# Add the scores as a new column in the DataFrame
pairs_small['ZS_counterspeech_score'] = counterspeech_scores
pairs_small['ZS_hate_score'] = hate_scores
pairs_small['ZS_neutral_score'] = neutral_scores


# Initialize the 'counterspeech' column with 'NA'
pairs_small['ZS_counterspeech'] = 'NA'

# Update the 'counterspeech' column based on the 'counterspeech_score'
for index, row in pairs_small.iterrows():
    if row['ZS_counterspeech_score'] > 0.8:
        pairs_small.at[index, 'ZS_counterspeech'] = 'yes'
    else:
        pairs_small.at[index, 'ZS_counterspeech'] = 'no'    


pairs_small

Extracting scores: 100%|████████████████████| 2000/2000 [06:34<00:00,  5.06it/s]
/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_20768/3435232829.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pairs_small['ZS_counterspeech_score'] = counterspeech_scores
/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_20768/3435232829.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pairs_small['ZS_hate_score'] = hate_scores
/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_20768/3435

,conversation_id,lang,created_at,id,text,author_id,replied_to_reply_count,referenced_tweets_id,in_reply_to_user_id,PNR,...,like_n,retweet_n,quote_n,translated,label_hate,score_hate,ZS_counterspeech_score,ZS_hate_score,ZS_neutral_score,ZS_counterspeech
0,1036583587242549248,da,2018-09-03 21:04:41,1036721666628444160,@frkomo Jeg siger det vel strengt taget bare t...,1666088336,1.0,1036721302692917250,148061237,1311570613,...,1,0,0,@frkomo I guess I'm saying it strictly just to...,normal,0.761725,0.632259,0.034136,0.333605,no
1,1036583587242549248,da,2018-09-03 21:03:14,1036721302692917250,"@MonbergSF Sig det til spillerforeningen, som ...",148061237,NaN,NaN,1666088336,NaN,...,1,0,0,"@MonbergSF Tell it to the gaming association, ...",normal,0.743716,0.951389,0.019291,0.029320,yes
2,899548260863488002,da,2017-08-21 12:10:42,899604671375052801,"@PeterHuggler Had alt det, du vil. Men du skal...",547416021,1.0,899603561323081729,3301029597,1405772015,...,0,0,0,@PeterHuggler Had everything you want. But don...,normal,0.733157,0.317035,0.016395,0.666571,no
3,899548260863488002,da,2017-08-21 12:06:17,899603561323081729,@brianweichardt Jeg hader den her slags: Du gå...,3301029597,NaN,NaN,547416021,NaN,...,1,0,0,@brianweichardt I hate this kind of thing: You...,normal,0.779667,0.095866,0.883837,0.020297,no
4,1345496479583113217,da,2021-01-03 12:38:16,1345711074407014400,@nielscallesoe @Heunicke Din første indvending...,87923613,1.0,1345524516311748608,23341699,0908801199,...,1,0,0,@nielsallesoe @Heunicke Your first objection m...,normal,0.738688,0.863429,0.016685,0.119886,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1285495153202024448,da,2020-07-21 10:22:06,1285520419529863168,@SimonStoerup @perlysholt Hm. Man ville som ud...,27626050,NaN,NaN,383396359,NaN,...,2,0,0,@SimonStoerup @perlysholt Hm. You would be abl...,normal,0.738122,0.768433,0.012006,0.219561,no
1996,1174395406719102976,da,2019-09-18 19:10:18,1174400270710820870,@R4nd4hl @khoenge Sjovt du synes netop Hønge b...,861057936,0.0,1174395406719102976,72823792,1508892043,...,2,0,0,@R4nd4hl @khoenge Funny you think just Hønge s...,normal,0.709584,0.917323,0.018060,0.064617,yes
1997,1174395406719102976,da,2019-09-18 18:50:58,1174395406719102976,Detektor har undersøgt det: @khoenge talte usa...,72823792,NaN,NaN,NaN,NaN,...,20,3,0,Detector has examined it: @khoenge spoke untru...,normal,0.786244,0.829699,0.078995,0.091306,yes
1998,1481298462238982144,da,2022-01-12 19:25:54,1481346717513662464,"@ReneAndersenDK Forskellen er, at ham her også...",805874425988087811,1.0,1481345545721495554,4776986009,NaN,...,2,0,0,@ReneAndersenDK The difference is that this gu...,normal,0.695681,0.834556,0.091175,0.074269,yes


In [ ]:
# on test data (danish), run

# Candidate labels
candidate_labels = ['counterspeech', 'hate', 'neutral tone']

# List to store the scores
counterspeech_scores = []
hate_scores = []
neutral_scores = []

# Iterate over each text
for text in (tqdm(pairs_small['text'], desc= "Extracting scores")) :
    result = pipe_1(text, candidate_labels)
    # Extract the scores
    counterspeech_score = dict(zip(result['labels'], result['scores']))['counterspeech']
    hate_score = dict(zip(result['labels'], result['scores']))['hate']
    neutral_score = dict(zip(result['labels'], result['scores']))['neutral tone']
    # Append the scores to the lists
    counterspeech_scores.append(counterspeech_score)
    hate_scores.append(hate_score)
    neutral_scores.append(neutral_score)

# Add the scores as a new column in the DataFrame
pairs_small['ZS_counterspeech_score_dan'] = counterspeech_scores
pairs_small['ZS_hate_score_dan'] = hate_scores
pairs_small['ZS_neutral_score_dan'] = neutral_scores


# Initialize the 'counterspeech' column with 'NA'
pairs_small['ZS_counterspeech'] = 'NA'

# Update the 'counterspeech' column based on the 'counterspeech_score'
for index, row in pairs_small.iterrows():
    if row['ZS_counterspeech_score_dan'] > 0.8:
        pairs_small.at[index, 'ZS_counterspeech_dan'] = 'yes'
    else:
        pairs_small.at[index, 'ZS_counterspeech_dan'] = 'no'    


pairs_small

In [44]:
#On full dataset

from tqdm import tqdm

# Candidate labels
candidate_labels = ['counterspeech', 'hate', 'neutral tone']

# List to store the scores
counterspeech_scores = []
hate_scores = []
neutral_scores = []

# Iterate over each text
for text in (tqdm(pairs['translated'], desc= "Extracting scores")) :
    result = pipe_1(text, candidate_labels)
    # Extract the scores
    counterspeech_score = dict(zip(result['labels'], result['scores']))['counterspeech']
    hate_score = dict(zip(result['labels'], result['scores']))['hate']
    neutral_score = dict(zip(result['labels'], result['scores']))['neutral tone']
    # Append the scores to the lists
    counterspeech_scores.append(counterspeech_score)
    hate_scores.append(hate_score)
    neutral_scores.append(neutral_score)

# Add the scores as a new column in the DataFrame
pairs['ZS_counterspeech_score'] = counterspeech_scores
pairs['ZS_hate_score'] = hate_scores
pairs['ZS_neutral_score'] = neutral_scores


# Initialize the 'counterspeech' column with 'NA'
pairs['ZS_counterspeech'] = 'NA'

# Update the 'counterspeech' column based on the 'counterspeech_score'
for index, row in pairs.iterrows():
    if row['ZS_counterspeech_score'] > 0.8:
        pairs.at[index, 'ZS_counterspeech'] = 'yes'
    else:
        pairs.at[index, 'ZS_counterspeech'] = 'no'    


pairs

Extracting scores:   4%|▋                 | 797/20000 [07:40<3:05:01,  1.73it/s]


KeyboardInterrupt: 

### cross-encoder/nli-deberta-base model

In [8]:
pipe_2 = pipeline("zero-shot-classification", model="cross-encoder/nli-deberta-base", device=0)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [9]:
candidate_labels = ['counterspeech', 'hate', 'neutral tone']

sequence_to_classify_1 = "i see your point but i dont think it is very constructive"
sequence_to_classify_2 = "jeg forstår hvad du mener, men din kommentar er ikke særlig konstruktiv"


pipe_2(sequence_to_classify_1, candidate_labels)

#doesnt seem very good

{'sequence': 'i see your point but i dont think it is very constructive',
 'labels': ['counterspeech', 'neutral tone', 'hate'],
 'scores': [0.5037503242492676, 0.4367963671684265, 0.05945330858230591]}

### TarsClassifier 

In [10]:
!pip install Flair


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [11]:
from flair.models import TARSClassifier
from flair.data import Sentence


tars = TARSClassifier.load('tars-base')


sentence = Sentence("jeg forstår hvad du mener, men din kommentar er ikke særlig konstruktiv")
sentence_2 = Sentence("i understand, but i dont agree")


classes = ["counterspeech", "hate"]

tars.predict_zero_shot(sentence_2, classes)

print(sentence_2)

2024-08-02 08:05:05,054 TARS initialized without a task. You need to call .add_and_switch_to_new_task() before training this model
2024-08-02 08:05:05,152 Provided `None` does not exist in the model. Consider calling `add_and_switch_to_new_task` first.
2024-08-02 08:05:05,153 `ZeroShot` is the current task. Switch to some other task before dropping this.
Sentence[7]: "i understand, but i dont agree"


## Applying classification models

#### Counterargument classifier (from huggingface)


https://huggingface.co/ThinkCERCA/counterargument_hugging

In [12]:
pipe_3 = pipeline("text-classification", model="ThinkCERCA/counterargument_hugging", device=0)

In [13]:
# Gives labels: Not Counterargument or Counterargument with a score

text = "I understand, but actually you are incorrect. Science show that the earth is round"
text_2 = "Jeg forstår din pointe, men den er ikke korrekt. Videnskaben viser at jorden er rund"
text_3 = "your a fool"
text_4 = "i like dogs, but also cats"

# Apply the pipeline to the text
result = pipe_3(text_4)
result

#allright?
#maybe test this on the test df

[{'label': 'Not Counterargument', 'score': 0.6886796355247498}]

In [74]:
# Test on small df (translated)

# List to store the scores
labels = []
scores = []

# Iterate over each text
for text in (tqdm(pairs_small['translated'], desc= "Extracting scores")) :
    result = pipe_3(text)
    # Extract the scores
    label = result[0]['label']
    score = result[0]['score']
    # Append the scores to the lists
    labels.append(label)
    scores.append(score)

# Add the scores as a new column in the DataFrame
pairs_small['coun_classifier_label'] = labels
pairs_small['coun_classifier_score'] = scores

pairs_small

Extracting scores: 100%|████████████████████| 2000/2000 [04:02<00:00,  8.23it/s]
/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_20768/2589889017.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pairs_small['coun_classifier_label'] = labels
/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_20768/2589889017.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pairs_small['coun_classifier_score'] = scores


,conversation_id,lang,created_at,id,text,author_id,replied_to_reply_count,referenced_tweets_id,in_reply_to_user_id,PNR,...,quote_n,translated,label_hate,score_hate,ZS_counterspeech_score,ZS_hate_score,ZS_neutral_score,ZS_counterspeech,coun_classifier_label,coun_classifier_score
0,1036583587242549248,da,2018-09-03 21:04:41,1036721666628444160,@frkomo Jeg siger det vel strengt taget bare t...,1666088336,1.0,1036721302692917250,148061237,1311570613,...,0,@frkomo I guess I'm saying it strictly just to...,normal,0.761725,0.632259,0.034136,0.333605,no,Not Counterargument,0.712887
1,1036583587242549248,da,2018-09-03 21:03:14,1036721302692917250,"@MonbergSF Sig det til spillerforeningen, som ...",148061237,NaN,NaN,1666088336,NaN,...,0,"@MonbergSF Tell it to the gaming association, ...",normal,0.743716,0.951389,0.019291,0.029320,yes,Not Counterargument,0.979896
2,899548260863488002,da,2017-08-21 12:10:42,899604671375052801,"@PeterHuggler Had alt det, du vil. Men du skal...",547416021,1.0,899603561323081729,3301029597,1405772015,...,0,@PeterHuggler Had everything you want. But don...,normal,0.733157,0.317035,0.016395,0.666571,no,Counterargument,0.686435
3,899548260863488002,da,2017-08-21 12:06:17,899603561323081729,@brianweichardt Jeg hader den her slags: Du gå...,3301029597,NaN,NaN,547416021,NaN,...,0,@brianweichardt I hate this kind of thing: You...,normal,0.779667,0.095866,0.883837,0.020297,no,Counterargument,0.825158
4,1345496479583113217,da,2021-01-03 12:38:16,1345711074407014400,@nielscallesoe @Heunicke Din første indvending...,87923613,1.0,1345524516311748608,23341699,0908801199,...,0,@nielsallesoe @Heunicke Your first objection m...,normal,0.738688,0.863429,0.016685,0.119886,yes,Counterargument,0.995546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1285495153202024448,da,2020-07-21 10:22:06,1285520419529863168,@SimonStoerup @perlysholt Hm. Man ville som ud...,27626050,NaN,NaN,383396359,NaN,...,0,@SimonStoerup @perlysholt Hm. You would be abl...,normal,0.738122,0.768433,0.012006,0.219561,no,Counterargument,0.851246
1996,1174395406719102976,da,2019-09-18 19:10:18,1174400270710820870,@R4nd4hl @khoenge Sjovt du synes netop Hønge b...,861057936,0.0,1174395406719102976,72823792,1508892043,...,0,@R4nd4hl @khoenge Funny you think just Hønge s...,normal,0.709584,0.917323,0.018060,0.064617,yes,Not Counterargument,0.564339
1997,1174395406719102976,da,2019-09-18 18:50:58,1174395406719102976,Detektor har undersøgt det: @khoenge talte usa...,72823792,NaN,NaN,NaN,NaN,...,0,Detector has examined it: @khoenge spoke untru...,normal,0.786244,0.829699,0.078995,0.091306,yes,Not Counterargument,0.975029
1998,1481298462238982144,da,2022-01-12 19:25:54,1481346717513662464,"@ReneAndersenDK Forskellen er, at ham her også...",805874425988087811,1.0,1481345545721495554,4776986009,NaN,...,0,@ReneAndersenDK The difference is that this gu...,normal,0.695681,0.834556,0.091175,0.074269,yes,Counterargument,0.767544


In [15]:
#check value counts
pairs_small['label'].value_counts(dropna=False) 


label
Not Counterargument    235
Counterargument         65
Name: count, dtype: int64

In [76]:
# look at performance

#pull up text examples
Counterargument = pairs_small[pairs_small['coun_classifier_label'] == 'Counterargument']
no_Counterargument = pairs_small[pairs_small['coun_classifier_label'] == 'Not Counterargument']


print(Counterargument['translated'][:100])

# can print the whole text using
text_to_print_1 = Counterargument['translated'].head(20).tolist()
text_to_print_1

print('no counterarguments:')
text_to_print_2 = no_Counterargument['translated'].head(20).tolist()
text_to_print_2

#


2      @PeterHuggler Had everything you want. But don...
3      @brianweichardt I hate this kind of thing: You...
4      @nielsallesoe @Heunicke Your first objection m...
5      @stinuslindgreen @Heunicke In my optics, it is...
6      @RasmusMalver @radical I'm sorry. I want to un...
                             ...                        
164    @sarahskarum @Trilsoe No, but neither do you, ...
166    @Petertwitdk @thomasmdom It is also a shame! W...
168    @boetter You just call/contact me Jacob, we ha...
170    @emiil...but your barren upper lip page, why i...
172    @Chrimajaki @BosseStine @RasmusJarlov No he ha...
Name: translated, Length: 100, dtype: object
no counterarguments:


["@frkomo I guess I'm saying it strictly just to those who follow me;) But if you follow the subsequent discussion, then I'm just not impressed by the approach the players have:)",
 '@MonbergSF Tell it to the gaming association, which is just fighting for DBU to have employer responsibility. Both with the gentlemen and the ladies',
 "Until an hour ago, @stinuslindgreen was on my top 5 over members of parliament. He is new in politics, but has really learned to bullshit. @Radio claimed that there was court control with access to private homes. There isn't. See how he tries to change direction. #dkpol https://t.co/37odfP9Y3O",
 "@ThomasMonbergSF I think we're running in a circle!... Have a good day and a go Pride!! • @BEsbensen @PiaOlsen",
 '@peterbrothersen Exactly... So thin post from @AnneFrostJepsen... "Don\'t beat the politicians even if they beat on a daily basis" Adult communication requires a desire from the politicians to act adultly and we still need to see',
 '@HellaSchulin @d

In [19]:
# Sort by 'score' in descending order
sorted_df = Counterargument.sort_values('score', ascending=False)

#print 
sorted_df['translated'].head(20).tolist()
#sorted_df[['translated', 'score', 'label']].head(10)

['@quitte74 @pferdinandsen Tusind tak, så har vi lidt at kigge på. Jeg ville selv have elsket de samme muligheder, men Basic på en Commodore 64 var nu heller ikke af vejen😉',
 '@KWEkristian Ah, det er da et lidt tyndt svar, når I selv har startet debatten - som du ved, har vi bestyrelsesmøde, så der bliver vist ikke megen regional deltagelse i den første del af jeres møde. @SigneLoentoft',
 '@quitte74 @PabloBresciani @ChrRothmann @stentoft01 @gratisbilletter @billetboers Jeg er vild med det man kan med kode, men har aldrig selv interesseret mig for hvordan det skal gøres. Så “vores” udvikler har lært at han enten skal sige “fint, det gør vi” eller “det kan du ikke få”, når jeg ønsker mig nye funktioner. Vi taler ikke længere om hvorfor 😂',
 '@Sklander89 @quitte74 Sklander for pokker. Du var selv direkte rasende over episoden, og nu beskylder du simpelthen mig for at lyve om det... 🤠 https://t.co/7GjV1bOVVO',
 '@JonasKGaba @alternativet_ @uffeelbaek Af de 7 påstande, vil jeg sige at 5 o

#### Bert counterspeech classifier (from huggingface)

Bert finetuned on the CONAN dataset

https://huggingface.co/tum-nlp/bert-counterspeech-classifier

In [20]:
pipe_4 = pipeline("text-classification", model="tum-nlp/bert-counterspeech-classifier",device=0)

In [23]:
text = "I understand, but actually you are incorrect. Science show that the earth is round"
text_2 = "Jeg forstår din pointe, men den er ikke korrekt. Videnskaben viser at jorden er rund"
text_3 = "you are a fool"

# Apply the pipeline to the text
result = pipe_4(text_2)
result

#Seems to work well on english data, not so good on danish data..

[{'label': 'non-counter-argument', 'score': 0.7306188941001892}]

#### Counterspeech score classifier (from huggingface)

https://huggingface.co/Hate-speech-CNERG/new-counterspeech-score

In [24]:
pipe_5 = pipeline("text-classification", model="Hate-speech-CNERG/new-counterspeech-score", device=0)

In [25]:
text = "I understand, but actually you are incorrect. Science show that the earth is round"
text_2 = "Jeg forstår din pointe, men den er ikke korrekt. Videnskaben viser at jorden er rund"
text_3 = "you are a fool"

# Apply the pipeline to the text
result = pipe_5(text_3)
result

# What labels, and what are the scores, test

[{'label': 'NEGATIVE', 'score': 0.7565722465515137}]

In [26]:
# Test on small df

# List to store the scores
labels_counter = []
scores_counter = []

# Iterate over each text
for text in (tqdm(pairs_small['translated'], desc= "Extracting scores")) :
    result = pipe_5(text)
    # Extract the scores
    label = result[0]['label']
    score = result[0]['score']
    # Append the scores to the lists
    labels_counter.append(label)
    scores_counter.append(score)

# Add the scores as a new column in the DataFrame
pairs_small['bert_coun_class_label'] = labels_counter
pairs_small['bert_coun_class_score'] = scores_counter

Extracting scores: 100%|██████████████████████| 300/300 [00:41<00:00,  7.29it/s]
/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_20768/1298344427.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pairs_test['label_counter'] = labels_counter
/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_20768/1298344427.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pairs_test['score_counter'] = scores_counter


In [27]:
pairs_small['bert_coun_class_label'].value_counts(dropna=False) 

label_counter
POSITIVE    180
NEGATIVE    120
Name: count, dtype: int64

In [28]:
positive = pairs_small[pairs_small['bert_coun_class_label'] == 'POSITIVE']


# Sort by 'score' in descending order
sorted_df = positive.sort_values('score', ascending=False)

#print 
sorted_df['translated'].head(20).tolist()
#sorted_df[['translated', 'score_counter', 'label_counter']].head(10)

['Min mors æblekage. Vi har prøvet at genlave den efter hun gik bort. Vi har hendes nedskrevne opskrift, men det smager bare ikke ens. Som hun nærmest har taget den med sig. Har I også sådan nogle retter/opskrifter i jeres familie?',
 '@olufsphere @AarhusGinger Vi som forældre har lidt hånd i hanke med hvad der er lavet / ikke er lavet + det samme med skolen. Det ser jeg som en ganske naturlig ting. \n\nJeg ved ikke hvorfor du vil blande den lockout ind i det her. jeg forholder mig udelukkende til den nuværende situation.',
 'For præcis elleve år siden kom jeg til København som håbefuld studerende (og aspirerende fodboldtræner).\n\nMeget er sket siden, men når jeg ser det her billede fra dengang, tænker jeg stadig:\n\n"Tænk, at en tysker kan have så meget charme, selvtillid og lækkert hår." https://t.co/aljCMBxZgu',
 '@PiaOlsen det respektere jeg 100%. Flot du tager dig tid til dialogen, politisk bliver vi nok ikke enige,men respekt for politisk engagement',
 '@lauralindahl Det er det 

## Applying hate classification model(s)

### HateXplain

In [ ]:
#Gives hate, offensive and normal 

In [29]:
# Use a pipeline as a high-level helper
pipe_6 = pipeline("text-classification", model="Hate-speech-CNERG/bert-base-uncased-hatexplain", device= 0)


In [38]:
text = "I understand, but actually you are incorrect. Science show that the earth is round"
text_2 = "Jeg forstår din pointe, men den er ikke korrekt. Videnskaben viser at jorden er rund"
text_3 = "you fat jewish nigga, i hate you"
text_4 = 'du er så tyk og grim'

# Apply the pipeline to the text
result = pipe_6(text_3)
result

#not so good on danish data?
#test

[{'label': 'offensive', 'score': 0.6110802292823792}]

In [70]:
# Test on small df


# List to store the scores
labels_hate = []
scores_hate = []

# Iterate over each text
for text in (tqdm(pairs_small['translated'], desc= "Extracting scores")) :
    result = pipe_6(text)
    # Extract the scores
    label = result[0]['label']
    score = result[0]['score']
    # Append the scores to the lists
    labels_hate.append(label)
    scores_hate.append(score)

# Add the scores as a new column in the DataFrame
pairs_small['label_hatexplain'] = labels_hate
pairs_small['score_hatexplain'] = scores_hate

Extracting scores: 100%|████████████████████| 2000/2000 [00:47<00:00, 42.45it/s]
/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_20768/3742457174.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pairs_small['label_hate'] = labels_hate
/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_20768/3742457174.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pairs_small['score_hate'] = scores_hate


In [52]:
hate

,conversation_id,lang,created_at,id,text,author_id,replied_to_reply_count,referenced_tweets_id,in_reply_to_user_id,PNR,...,counterspeech_score,hate_score,neutral_score,counterspeech,label,score,label_counter,score_counter,label_hate,score_hate


In [71]:
# look at performance

#pull up text examples
hate = pairs_small[pairs_small['label_hate'] == 'hate']
offensive = pairs_small[pairs_small['label_hate'] == 'offensive']
normal = pairs_small[pairs_small['label_hate'] == 'normal']


print(hate['translated'][:100])

# can print the whole text using
print('hate:')
text_to_print = hate['translated'].head(20).tolist()
print(text_to_print)

print('offensive:')
text_to_print = offensive['translated'].head(20).tolist()
text_to_print

Series([], Name: text, dtype: object)
hate:
[]
offensive:


['@BosseStine @RasmusJarlov Jarlov is quite right. However, politically correct halal radicals are willing to deny the truth.',
 '@MivJensen @provinsmor @Kong_Kret My family uses it intensively. I have told them that if they want me something important, then it is called messenger. Snapchat is the worst scum app ever, I can almost burn together over the shit •',
 '@Dittemedmening It\'s not "to talk to racists" who have right-turned the community. It\'s the racists who have done it. Because they want to be heard. And besides, I don\'t do that shit with "we who stand up to racism" I fucking do that too, even though I might do it differently than you do',
 '@lauravilsbaek I simply think it\'s part of the gamer culture. I can read elsewhere that it\'s nothing new. Discover it to football or wherever else they go - fortunately. But "fucking creepy faggot", "suck my... your whore" etc. - then I see red.',
 '@HoghSorensen @Wirlander @Cihat_Bardak @a_esbech @DanskDf1995 Learn a new word? I do 

### Hate roberta

In [78]:
pipe_7 = pipeline("text-classification", model="facebook/roberta-hate-speech-dynabench-r4-target", device= 0)



config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/151 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
text = "I understand, but actually you are incorrect. Science show that the earth is round"
text_2 = "Jeg forstår din pointe, men den er ikke korrekt. Videnskaben viser at jorden er rund"
text_3 = "you are a fool"


# Apply the pipeline to the text
result = pipe_6(text_3)
result

## Finetuning a model on labeled counterspeech data (CONAN)

CONAN (COunter NArratives through Nichesourcing): A dataset containing counterspeech responses to hate speech. Data is French, italian and English

https://github.com/marcoguerini/CONAN

https://aclanthology.org/P19-1271.pdf

## Applying the jigsaw perspective API
